In [7]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Flatten, Dense
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
import keras

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

In [4]:
img_width, img_height = 16, 16

nb_epoch = 50
batch_size = 32

In [21]:
base_dir = '/home/niaki/Code/ImageNet/tiny-imagenet-200'

train_data_dir      = base_dir + '/tiny_train16'
validation_data_dir = base_dir + '/tiny_validation16'
test_data_dir       = base_dir + '/tiny_test16'

train_descrs_dir      = base_dir + '/tiny_sifts/tiny_train16'
validation_descrs_dir = base_dir + '/tiny_sifts/tiny_validation16'
test_descrs_dir       = base_dir + '/tiny_sifts/tiny_test16'


# nb_train_samples = 0
# train_folders = os.listdir(train_data_dir)
# train_folders.sort()
# for folder in train_folders:
#     nb_train_samples += len(os.listdir(train_data_dir + '/' + folder))
# print(nb_train_samples)

# nb_validation_samples = 0
# validation_folders = os.listdir(validation_data_dir)
# validation_folders.sort()
# for folder in validation_folders:
#     nb_validation_samples += len(os.listdir(validation_data_dir + '/' + folder))
# print(nb_validation_samples)

In [16]:
def loading_data(data_dir):
    files = listdir(data_dir + '/class0')
    files.sort()

    images = []

    for file in files:
        image = imageio.imread(data_dir + '/class0/' + file)
    #     image = np.expand_dims(image, axis=0)
        images.append(image)

    images = np.array(images)
    images = images.astype(np.float64) / 255
    print(images.shape)
    return images

In [19]:
x_train = loading_data(train_data_dir)
x_validation = loading_data(validation_data_dir)

(157086, 16, 16, 3)
(3932, 16, 16, 3)


In [25]:
def loading_descrs(descrs_dir):
    files = listdir(descrs_dir + '/class0')
    files.sort()

    descrs = []

    for file in files:
        descr = np.load(descrs_dir + '/class0/' + file)
        descrs.append(descr)

    descrs = np.array(descrs)
    print(descrs.shape)
    return descrs

In [27]:
y_train = loading_descrs(train_descrs_dir)
y_validation = loading_descrs(validation_descrs_dir)

(157086, 128)
(3932, 128)


In [11]:
input_shape = (img_width, img_height, 3)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(128, activation="relu")(x)

encoder = Model(input_img, encoded)

encoder.compile(optimizer='adadelta', loss='binary_crossentropy')

encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16, 16, 3)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 32)        896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 8, 8, 32)          9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 4, 4, 32)          9248      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 2, 2, 32)          0         
__________

In [28]:
encoder.fit(x_train, y_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_validation, y_validation))


Train on 157086 samples, validate on 3932 samples
Epoch 1/50
157086/157086 [==============================] - 125s 798us/step - loss: -212.4877 - val_loss: -217.3632
Epoch 2/50
157086/157086 [==============================] - 129s 821us/step - loss: -218.7053 - val_loss: -217.3632
Epoch 3/50
157086/157086 [==============================] - 131s 834us/step - loss: -218.7053 - val_loss: -217.3632
Epoch 4/50
157086/157086 [==============================] - 127s 806us/step - loss: -218.7053 - val_loss: -217.3632
Epoch 5/50
157086/157086 [==============================] - 126s 801us/step - loss: -218.7053 - val_loss: -217.3632
Epoch 6/50
157086/157086 [==============================] - 126s 800us/step - loss: -218.7053 - val_loss: -217.3632
Epoch 7/50
157086/157086 [==============================] - 125s 796us/step - loss: -218.7053 - val_loss: -217.3632
Epoch 8/50
157086/157086 [==============================] - 126s 804us/step - loss: -218.7053 - val_loss: -217.3632
Epoch 9/50
157086/1570